# TOOL MCGVYERING: TOOL CREATION WITH AUXILARY OBJECTS


## Required Libraries & Parameters

In [1]:
import sys
sys.path.append('../src')
import numpy as np
import robotic as ry

from RAI import RAI
from SEG import SEG
from SQF import SQF
from Score import Score

cam_list = ["cam_front_left", "cam_front_right", "cam_back_right", "cam_back_left", "cam_up", "cam_front", "cam_left", "cam_back", "cam_right"] 
filter = 1
model_path = "../src/models/tools/simple/parts"
target_path = "../src/point_clouds_ref/"
scene = ry.Config()
scene.addFile("../src/config/tools_simple3_s.g")
ref_file = "../src/parameters_ref.csv"
cand_file = "../src/parameters_cand.csv"

reference_tool = "spoon" # shovel, hammer, rake # Select a reference tool

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## STEP 1: Generate the point cloud for the reference tool

In [2]:
print("Generating the reference point cloud")
rai = RAI(verbose=0)
segments_ref = rai.generate_ref_point_cloud(model_path, cam_list, filter = filter, object_name=reference_tool)

Generating the reference point cloud
Model name: spoon_head
Model name: spoon_body


## STEP 2: Generate the point cloud for the candidate objects

In [3]:
print("Generating the candidate point cloud")
ptc_cand, _, _ = rai.get_raw_point_cloud(scene, cam_list[0:7], filter = filter)

Generating the candidate point cloud


## STEP 3: Remove the plane from the scene

In [4]:
print("Removing the plane from the scene")
seg = SEG(verbose=0)
pcl_filtered = seg.RANSAC_plane(ptc_cand)

print("Segmenting the objects")
segments_cand = seg.segment_objects(pcl_filtered, scene)

Removing the plane from the scene
Segmenting the objects


## STEP 4: Fit SuperQuadric function to the objects

In [6]:
param_ref_full = SQF.load_parameters_from_csv(ref_file)
param_cand_full = SQF.load_parameters_from_csv(cand_file)

param_ref = {key: param_ref_full[key] for key in segments_ref.keys() if key in param_ref_full}
param_cand = {key: param_cand_full[key] for key in segments_cand.keys() if key in param_cand_full}

missing_ref_keys = set(segments_ref.keys()) - set(param_ref.keys())
if missing_ref_keys:
    print(f"Fitting SuperQuadric function for missing reference objects: {missing_ref_keys}")
    for key in missing_ref_keys:
        pcd = segments_ref[key]
        sqf = SQF(pcd, verbose=0)
        try:
            param, _, _, _ = sqf.fit(sq_type=[0])
            if len(param) < 12:
                param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
            param_ref[key] = param
        except ValueError as e:
            print(f"Error fitting SQF for reference object {key}: {e}")
    #SQF.save_parameters_to_csv({**param_ref_full, **param_ref}, ref_file)

missing_cand_keys = set(segments_cand.keys()) - set(param_cand.keys())
if missing_cand_keys:
    print(f"Fitting SuperQuadric function for missing candidate objects: {missing_cand_keys}")
    for key in missing_cand_keys:
        pcd = segments_cand[key]
        sqf = SQF(pcd, verbose=0)
        try:
            param, _, _, _ = sqf.fit(sq_type=[0])
            if len(param) < 12:
                param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
            param_cand[key] = param
        except ValueError as e:
            print(f"Error fitting SQF for candidate object {key}: {e}")
    #SQF.save_parameters_to_csv({**param_cand_full, **param_cand}, cand_file)


Fitting SuperQuadric function for missing reference objects: {'spoon_head', 'spoon_body'}
`ftol` termination condition is satisfied.
Function evaluations 162, initial cost 1.5680e+01, final cost 2.7621e-01, first-order optimality 3.45e-03.


/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:334: RuntimeWarning: overflow encountered in power
  Z1 = Z ** (2 / eps1)
/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:337: RuntimeWarning: overflow encountered in power
  quadric_func = (X1 + Y1) ** (eps2 / eps1) + Z1


`ftol` termination condition is satisfied.
Function evaluations 264, initial cost 2.6141e+01, final cost 9.2429e-05, first-order optimality 2.37e-06.
Fitting SuperQuadric function for missing candidate objects: {'obj_prism_short', 'obj_cylinder', 'obj_cube_hollow', 'obj_sphere_half_hollow'}
`ftol` termination condition is satisfied.
Function evaluations 228, initial cost 2.1550e+01, final cost 5.3557e-04, first-order optimality 1.39e-05.


/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:332: RuntimeWarning: overflow encountered in power
  X1 = X ** (2 / eps2)
/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:333: RuntimeWarning: overflow encountered in power
  Y1 = Y ** (2 / eps2)


`ftol` termination condition is satisfied.
Function evaluations 255, initial cost 1.6461e+01, final cost 3.9793e-04, first-order optimality 1.56e-05.


/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:332: RuntimeWarning: overflow encountered in power
  X1 = X ** (2 / eps2)
/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:333: RuntimeWarning: overflow encountered in power
  Y1 = Y ** (2 / eps2)
/home/bora/Bilkent_MS/CS554_Computer_Vision/CS554_Computer_Vision_Project/test/../src/SQF.py:334: RuntimeWarning: overflow encountered in power
  Z1 = Z ** (2 / eps1)


`ftol` termination condition is satisfied.
Function evaluations 202, initial cost 4.7369e+01, final cost 2.7905e-02, first-order optimality 3.62e-04.
`ftol` termination condition is satisfied.
Function evaluations 420, initial cost 3.9205e+00, final cost 8.3585e-02, first-order optimality 8.46e-05.


## STEP 5: Find the best matching candidates objects to the reference tool set

In [7]:
print("Finding the best matching candidate objects to the reference tool set")
score = Score(param_ref, param_cand, verbose=1)
match_1, match_2 = score.algorithm_1()

Finding the best matching candidate objects to the reference tool set
Matches: [('spoon_head', 'obj_sphere_half_hollow'), ('spoon_body', 'obj_prism_short')], Total Error: 0.5234, Shape Error: 0.0810, Size Error: 0.2330, Ratio Error: 0.4188
Matches: [('spoon_head', 'obj_sphere_half_hollow'), ('spoon_body', 'obj_cube_hollow')], Total Error: 1.3233, Shape Error: 0.0830, Size Error: 0.6681, Ratio Error: 1.1444
Matches: [('spoon_head', 'obj_sphere_half_hollow'), ('spoon_body', 'obj_cylinder')], Total Error: 1.6403, Shape Error: 1.0419, Size Error: 0.3360, Ratio Error: 0.5247
Matches: [('spoon_head', 'obj_cylinder'), ('spoon_body', 'obj_prism_short')], Total Error: 1.9839, Shape Error: 0.9102, Size Error: 0.6306, Ratio Error: 0.8864
Matches: [('spoon_head', 'obj_cube_hollow'), ('spoon_body', 'obj_prism_short')], Total Error: 2.1512, Shape Error: 1.3242, Size Error: 0.5224, Ratio Error: 0.6094
Matches: [('spoon_head', 'obj_cylinder'), ('spoon_body', 'obj_cube_hollow')], Total Error: 2.7838, S

## STEP 6: Create the tool

In [ ]:
rai.create_tool(scene, match_1, match_2)